# Libraries and Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [3]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.8/784.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.0/275.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2

In [4]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid

In [5]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv', index_col = 0, parse_dates = True)
future_df = pd.read_csv('future.csv', index_col = 0, parse_dates = True)

In [7]:
future_df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2021-01-01,NaN,0,0,0,5.00,154.221
2021-01-02,NaN,0,0,0,11.11,264.805
2021-01-03,NaN,0,0,0,3.89,115.499
2021-01-04,NaN,0,0,0,6.67,124.650
2021-01-05,NaN,0,0,0,5.56,77.968


In [8]:
# Extract the regressors
X_train = df.iloc[:,1:]
X_future = future_df.iloc[:,1:]

# Merge the 2 inputs
X = pd.concat([X_train, X_future])

In [9]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


# Prepare for LSTM

In [10]:
# Creating Time Series
from darts.timeseries import TimeSeries
series = TimeSeries.from_series(df.y)
covariates = TimeSeries.from_dataframe(X)

In [18]:
# Seasonality / trend
from darts.utils.timeseries_generation import datetime_attribute_timeseries

# Year
year_series = datetime_attribute_timeseries(
    pd.date_range(start = series.start_time(),
                  freq = series.freq_str,
                  periods = X.shape[0]),
    attribute = "year",
    one_hot = False)

# Month
month_series = datetime_attribute_timeseries(year_series,
                                             attribute= "month",
                                             one_hot = True)

# Weekday
weekday_series = datetime_attribute_timeseries(year_series,
                                             attribute= "weekday",
                                             one_hot = True)


In [19]:
# Scaling
from darts.dataprocessing.transformers import Scaler
transformer1 = Scaler()
transformer2 = Scaler()

In [20]:
# Scaling Y
y_transformed = transformer1.fit_transform(series)

In [21]:
# Stacking the covariates
covariates = covariates.stack(year_series)

In [22]:
# Scaling the Covariates
covariates_transformed = transformer2.fit_transform(covariates)

In [23]:
# Stack the Seasonal variables
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

# LSTM Model

In [24]:
# Load the tuned parameters
parameters = pd.read_csv("Forecasting Product/best_params_lstm.csv",
                         index_col = 0)
parameters

,12
dropout,0.100000
hidden_dim,20.000000
input_chunk_length,20.000000
lr,0.003000
n_epochs,10.000000
n_rnn_layers,1.000000
training_length,20.000000
rmse,95.303638


In [25]:
# Isolate the params
n_rnn_layers = int(parameters.loc['n_rnn_layers'])
hidden_dim = int(parameters.loc['hidden_dim'])
dropout = float(parameters.loc['dropout'])
lr = float(parameters.loc['lr'])
input_chunk_length = int(parameters.loc['input_chunk_length'])
n_epochs = int(parameters.loc['n_epochs'])
training_length = int(parameters.loc['training_length'])

In [26]:
# Build the LSTM model
# https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html
from darts.models import RNNModel
model = RNNModel(model = "LSTM",
                 hidden_dim = hidden_dim,
                 n_rnn_layers = n_rnn_layers,
                 dropout = dropout,
                 n_epochs = n_epochs,
                 optimizer_kwargs = {"lr": lr},
                 random_state = 1502,
                 training_length = training_length,
                 input_chunk_length=input_chunk_length,
                 pl_trainer_kwargs = {"accelerator": "gpu", "devices": [0]}
                 )

In [27]:
# Fit the model to the data
model.fit(y_transformed,
          future_covariates = covariates_transformed)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rn

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


RNNModel(model=LSTM, hidden_dim=20, n_rnn_layers=1, dropout=0.1, training_length=20, n_epochs=10, optimizer_kwargs={'lr': 0.003}, random_state=1502, input_chunk_length=20, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]})

# Forecasting and Exporting

In [33]:
predictions_lstm = model.predict(n = len(future_df),
                                 future_covariates = covariates_transformed)
predictions_lstm = TimeSeries.pd_series(
    transformer1.inverse_transform(
        predictions_lstm)).rename("lstm")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [34]:
# Exporting
predictions_lstm.to_csv("Forecasting Product/Ensemble/predictions_lstm.csv")